In [ ]:
# pip install -r requirements.txt
# pip install pandas

In [2]:
# import zipfile

# with zipfile.ZipFile('./final_dataset.zip', 'r') as zip_ref:
#     zip_ref.extractall('./')

In [1]:
import math
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image, UnidentifiedImageError
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer, AutoConfig

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    try:
        image = Image.open(image_file).convert('RGB')
    except UnidentifiedImageError:
        print(f"Warning: Cannot identify image file {image_file}, skipping.")
        return None
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers
    # Since the first GPU will be used for ViT, treat it as half a GPU.
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.model.rotary_emb'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

# If you set `load_in_8bit=True`, you will need two 80GB GPUs.
# If you set `load_in_8bit=False`, you will need at least three 80GB GPUs.
path = 'OpenGVLab/InternVL3-14B-Instruct'
device_map = split_model('OpenGVLab/InternVL3-14B-Instruct')
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=False,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
    device_map=device_map).eval()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

FlashAttention2 is not installed.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# # set the max number of tiles in `max_num`
# # pixel_values = load_image('./b3d57c7920d2d497d2b0d48f8671a899107ca903d88a9798d6a79f77.jpg', max_num=12).to(torch.bfloat16).cuda()
# pixel_values = load_image('./train/images/0a0da4831d09c0fa38980b4fbc74074735f69068223f42ccac480936.jpg', max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=1024, do_sample=True)
# # pure-text conversation (纯文本对话)
# question = 'Hello, who are you?'
# response, history = model.chat(tokenizer, None, question, generation_config, history=None, return_history=True)
# print(f'User: {question}\nAssistant: {response}')
# # single-image single-round conversation
# question = '<image>\nThis image has not been modified or edited from the original through certain means, either technical or manual, and it is not a synthetic image. It appears natural and realistic." Please explain why clearly and concisely in less than 100 words.'
# response = model.chat(tokenizer, pixel_values, question, generation_config)
# print(f'User: {question}\nAssistant: {response}')

In [24]:
def normalize_content_manipulation(value):
    if isinstance(value, bool):
        return value
    if isinstance(value, str):
        value = value.strip().lower() 
        if value == 'true': 
            return True
        elif value == 'false': 
            return False
    return None
    
def generate_prompt(image_folder, image_id, content_manipulation):
    img_path = os.path.join(image_folder, f'{image_id}.jpg')
    print(img_path)
    print(content_manipulation, type(content_manipulation))
    if os.path.exists(img_path):
        pixel_values = load_image(img_path, max_num=12)
        if pixel_values is None:
            return None
        pixel_values = pixel_values.to(torch.bfloat16).cuda()
        content_manipulation = normalize_content_manipulation(content_manipulation)
        if content_manipulation is None:
            del pixel_values
            torch.cuda.empty_cache()
            return None
        if content_manipulation:
            question_TRUE = '<image>\nThis image may have been modified or edited from the original through certain means, either technical or manual, or it may itself be a synthetic image, making it appear unnatural or unrealistic. Please briefly and clearly tell me the most possible reason in no more than 100 words.'
            response_TRUE = model.chat(tokenizer, pixel_values, question_TRUE, generation_config)
            print(f'Assistant: {response_TRUE}')
            del pixel_values
            torch.cuda.empty_cache()
            return response_TRUE
        if content_manipulation == 'false' or 'FALSE' or False:
            question_FALSE = '<image>\nThis image has not been modified or edited from the original through certain means, either technical or manual, and it is not a synthetic image. It appears natural and realistic." Please explain why clearly and concisely in less than 100 words.'
            response_FALSE = model.chat(tokenizer, pixel_values, question_FALSE, generation_config)
            print(f'Assistant: {response_FALSE}')
            del pixel_values
            torch.cuda.empty_cache()
            return response_FALSE
    else:
        return None 

In [ ]:
import pandas as pd
import os

train = pd.read_csv('./train/labels.csv')
test = pd.read_csv('./test/labels.csv')
val = pd.read_csv('./val/labels.csv')

# data = pd.concat([train, test, val], ignore_index=True)

train = train.drop_duplicates(subset='image_id', keep='first')
test = test.drop_duplicates(subset='image_id', keep='first')
val = val.drop_duplicates(subset='image_id', keep='first')

train['prompts'] = ''
test['prompts'] = ''
val['prompts'] = ''

train_images_folder = os.path.join(os.path.dirname('./train/labels.csv'), 'images')
test_images_folder = os.path.join(os.path.dirname('./test/labels.csv'), 'images')
val_images_folder = os.path.join(os.path.dirname('./val/labels.csv'), 'images')

train['prompts'] = train.apply(lambda row: generate_prompt(train_images_folder, row['image_id'], row['content_manipulation']), axis=1)
test['prompts'] = test.apply(lambda row: generate_prompt(test_images_folder, row['image_id'], row['content_manipulation']), axis=1)
val['prompts'] = val.apply(lambda row: generate_prompt(val_images_folder, row['image_id'], row['content_manipulation']), axis=1)

train.to_csv('./train/labels_new.csv', index=False)
test.to_csv('./test/labels_new.csv', index=False)
val.to_csv('./val/labels_new.csv', index=False)

In [ ]:
train = pd.read_csv('./train/labels_new.csv')
test = pd.read_csv('./test/labels_new.csv')
val = pd.read_csv('./val/labels_new.csv')

train['prompts'] = train.apply(
    lambda row: generate_prompt(train_images_folder, row['image_id'], row['content_manipulation']) 
    if pd.isnull(row['prompts']) or row['prompts'] == '' 
    else row['prompts'], axis=1
)

In [26]:
test['prompts'] = test.apply(
    lambda row: generate_prompt(test_images_folder, row['image_id'], row['content_manipulation']) 
    if pd.isnull(row['prompts']) or row['prompts'] == '' 
    else row['prompts'], axis=1
)

val['prompts'] = val.apply(
    lambda row: generate_prompt(val_images_folder, row['image_id'], row['content_manipulation']) 
    if pd.isnull(row['prompts']) or row['prompts'] == '' 
    else row['prompts'], axis=1
)

train.to_csv('./train/labels_new_2.csv', index=False)
test.to_csv('./test/labels_new_2.csv', index=False)
val.to_csv('./val/labels_new_2.csv', index=False)

./test/images/74db6bbec56f371834305d0960e109cf8ade559fbdc4f85013f89c2b.jpg
True <class 'str'>


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Assistant: The image appears to have been digitally altered. The person has a distorted, elongated appearance, likely due to scaling or warping. This modification disrupts the natural proportions, giving an unrealistic look. The background elements are out of focus or unevenly lit, adding to the impression of a synthetic image. Overall, these factors suggest that digital manipulation has been used to create the unnatural effect.
./test/images/89f2e1cdfb74a73e7c22a7dc646acd087cd75518119b9478b3657550.jpg
True <class 'str'>
Assistant: The image appears to have been digitally altered. The text on the t-shirt, combined with the context of the scene, suggests manipulation for humorous or satirical purposes. The man's expression and shirt text seem inconsistent with realistic attire choices in public settings, indicating editing.
./test/images/27ee4e9249011ed2cdb194ff911219dee858e371904b4ceeeec8235a.jpg
False <class 'str'>


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Assistant: The image appears natural and realistic due to its clear lighting, realistic shadows, and the presence of detailed objects and settings typical in such contexts – participants in formal attire, a setting with decor and furniture suggestive of a conference or official meeting. The composition is coherent and consistent with real-life conventions of photojournalism or official documentation, showing unaltered, genuine participation and engagement. This indicates the image is likely authentic, unaltered through technological or manual means.
./test/images/92d4b3f2b2578fe51e15ab03055249d5d86001aa18b0c0b2c85f5dc2.jpg
True <class 'str'>


...

Assistant: This image appears realistic because it shows a natural maritime scene. The ships are depicted in a way that matches real naval vessels' characteristics, such as structure, equipment, and color. The perspective and lighting suggest a genuine ocean environment. The text is clear and matches known formatting styles; the flag indicates Australia. There are no visible signs of digital manipulation, such as mismatched textures or awkward blending. Therefore, it seems unedited and synthetic-free.
./val/images/4f93c4ba1920d36da6842c4f1fe74a2f122a31be4e5a290f91e016f3.jpg
False <class 'bool'>


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Assistant: The image appears natural and realistic because there are no obvious signs of editing or manipulation. The lighting, shadows, and overall composition cohere, with natural transitions between subjects and background elements. The individuals’ features and clothing textures look consistent with real-life photographs, lacking any artificial or disjointed aspects. There are no visible artifacts, overlaps, or inconsistencies that would suggest digital alteration. Additionally, the background includes realistic details, such as household items and decor, further indicating authenticity.
./val/images/3769c863e70d1969f5e2338e6946881e803ecd1ef56e63b983555891.jpg
False <class 'bool'>


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Assistant: This image appears natural and realistic because it looks unaltered. The lighting, shadows, and reflections are consistent with a real scene. The people's poses and expressions also appear genuine, and there are no obvious signs of digital manipulation like mismatched edges or distorted features. Additionally, the background elements, such as the castle and people in the distance, are naturally integrated with the foreground. This all suggests that the image was taken as-is without editing.
./val/images/f9bf6a158c2f8e1d6e8385ae59920247685b4653c12615e363543518.jpg
False <class 'bool'>
Assistant: This image seems to be realistic because it shows a natural-looking human hand unboxing a device, with recognizable features like a visible ring, short nail polish, and realistic text on the note. The background elements, such as the shredded yellow paper, complement the scene and add to its authenticity. There's no indication of unnatural edits or synthetic qualities that would sugge

In [27]:
train = train[train['prompts'].notna() & (train['prompts'] != '')]

In [28]:
test = test[test['prompts'].notna() & (test['prompts'] != '')]
val = val[val['prompts'].notna() & (val['prompts'] != '')]

train.to_csv('./train/labels_new_3.csv', index=False)
test.to_csv('./test/labels_new_3.csv', index=False)
val.to_csv('./val/labels_new_3.csv', index=False)